In [1]:
import numpy as np

In [2]:
import sys
import os
import glob
import torch
import matplotlib.pyplot as pltm
sys.path.append("../")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from configs.wav2vec2_characterwise_pretrained_ctc_augs import Configs
CFG = Configs()

/usr/local/lib/python3.8/dist-packages/torch/storage.py:315: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.
  warnings.warn(message, UserWarning)


length of train: 100, length of valid: 100


In [9]:
from trainer.wav2vec2_fintune_ctc_trainer import Trainer

In [10]:
CFG.DISTRIBUTED=False
train = Trainer(CFG)

freezing encoder layer....
Using collate function..
Autoregressive inference: False


In [11]:
loader = iter(train.train_loader)

In [19]:
z = next(loader)
[tt.shape for tt in z]

[torch.Size([12, 126720]),
 torch.Size([12, 95]),
 torch.Size([12]),
 torch.Size([12])]

In [23]:
z[3]

tensor([74, 71, 19, 37, 48, 80, 29, 71, 85, 95, 35, 67])

In [24]:
126720/320

396.0

In [26]:
CFG.model.cuda()
1

1

In [51]:
with torch.no_grad():
    sn=399999
    pd = CFG.model(torch.randn(1,sn).cuda())
    print(sn,pd.shape,sn//320)

399999 torch.Size([1, 1249, 81]) 1249


In [50]:
(sn-1)/320

313.246875